In [1]:
import numpy as np
import pandas as pd

In [2]:
# Load tidied dataframes

# spells_df = pd.read_pickle('data/spells_df')
monster_df = pd.read_pickle('data/monster_df')
actions_df = pd.read_pickle('data/actions_df')
spellbook_df = pd.read_pickle('data/spellbook_df')
innate_spellbook_df = pd.read_pickle('data/innate_spellbook_df')
# special_abilities_df = pd.read_pickle('data/special_abilities_df')
# condition_immunities_df = pd.read_pickle('data/condition_immunities_df')
# damage_immunities_df = pd.read_pickle('data/damage_immunities_df')
# damage_resistances_df = pd.read_pickle('data/damage_resistances_df')
# damage_vulnerabilities_df = pd.read_pickle('data/damage_vulnerabilities_df')

# `actions_df`

In [3]:
actions_df = actions_df.reset_index('action')

In [4]:
outliers = ['Animated Armor',
            'Cult Fanatic',
            'Deva',
            'Doppelganger',
            'Erinyes',
            'Gibbering Mouther',
            'Gladiator',
            'Iron Golem',
            'Knight',
            'Lizardfolk',
            'Planetar',
            'Scout',
            'Spy',
            'Thug',
            'Violet Fungus',
            'Werebear']

In [5]:
a = (actions_df.index).difference(outliers)
df = actions_df.loc[a]

In [46]:
def normalize_multiattack(x):
    df = x.set_index('action').copy()
    if 'Multiattack' in df.index:
        desc = df.loc['Multiattack'].desc.lower().strip()
        actions = [format_action(action) for action in df.index
                   if action != 'Multiattack']
        print(x.name)
        print(actions)
        for line in desc.split('. '):
            extract_attacks(line, actions)

def format_action(action):
    action = action.lower()
    i = action.find('(')
    if i > 0:
        return action[:i]
    else:
        return action
        
        
            
def extract_attacks(line, actions):
    if ':' in line:
#         print('\t', line)
#         print('\t', [])
        pass
    else:
        print('\t', line)
        print('\t', [action for action in actions if action in line])
    return
            
numerics = {'one': 1,
            'two': 2,
            'three': 3,
            'six': 6,
            'seven': 7,
            'twice': 2}

In [47]:
'with its', 'or', 'alternatively'

('with its', 'or', 'alternatively')

In [48]:
df.groupby('name').apply(normalize_multiattack)

Aboleth
['tentacle', 'tail', 'enslave ']
	 the aboleth makes three tentacle attacks.
	 ['tentacle']
Aboleth
['tentacle', 'tail', 'enslave ']
	 the aboleth makes three tentacle attacks.
	 ['tentacle']
Adult Black Dragon
['bite', 'claw', 'tail', 'frightful presence', 'acid breath ']
	 the dragon can use its frightful presence
	 ['frightful presence']
Adult Blue Dracolich
['bite', 'claw', 'tail', 'frightful presence', 'lightning breath ']
	 the dracolich can use its frightful presence
	 ['frightful presence']
Adult Blue Dragon
['bite', 'claw', 'tail', 'frightful presence', 'lightning breath ']
	 the dragon can use its frightful presence
	 ['frightful presence']
Adult Brass Dragon
['bite', 'claw', 'tail', 'frightful presence', 'breath weapons ']
	 the dragon can use its frightful presence
	 ['frightful presence']
Adult Bronze Dragon
['bite', 'claw', 'tail', 'frightful presence', 'breath weapons ']
	 the dragon can use its frightful presence
	 ['frightful presence']
Adult Copper Dragon
['bi

""
